## Zero-order methods

#### Problem:
$$
f(\vec{x}) \rightarrow min,\\
f: \Omega \rightarrow \mathbb{R}, \\
\vec{x_*} \in \Omega, f_{min} = f(\vec{x_*})
$$

In zero-order methods we use only information about function value in point. Many of these methods are not theoretically justified and are built heoristicly. So questions about convergences are comonly absent. But they are in connection with first- and second-order methods, so the way to measure efficiency is comparsion.

#### Features of the zero-order methods

Let's describe a simple algorithm of zero-order method. Let us have $f: \mathbb{R}^n \rightarrow \mathbb{R}$. We pick point $\vec{x}_0$ which is called *base*. Than we evaluate $f(\vec{x}_0)$ and build n-dimension cube with length $l$ and center in this point. Set of edges of this cube is named *pattern*. We evaluate function in all points from pattern and then we pick the minimum and the procedure continiues with some reduction of length $l$.

The drawback of this method is obvious. Our points can repeat, to avoid this, we can use *simplexes*. ***Simplex*** is an $n+1$-dimensional convex polytope.

**Regular simplex** is a simplex with all equaldistant edges. J. Nelder and R. Meald first introduced *irregular simplexes*, which are basis of *Controlled direct search* or *Nelder-Mead algorithm*

In [139]:
import numpy as np
from numpy.linalg import norm
from test_functions import sphere, himmelblau, bukin

### Nelder-Mead Algorithm

Let $S_k \subset \mathbb{R^n}$ is true simplex. The procedure of finding next simplex, in wich function has less value consists of several stages: **reflection** of $\vec{x}_{l,n+1}$, **transforming** size and form of simplex if search step is sucessful or simplex **reduction** if not.

Vertex reflection of $\vec{x}_{k+1,n+1}$ using this formula:
$$
\vec{x}_{k+1,n+1} = \frac{1+\alpha}{n}\sum^n_{i=1}\vec{x}_{k,i} - \alpha \vec{x}_{k,n+1}
$$
Where $\alpha$ is a reflection coefficient. $\sum^n_{i=1}\vec{x}_{k,i}$ - center of edge

If in new vertex the value of function is less, we have success and there goes the procedure of scaling size and form of simplex $S_{k+1}$. 

If $f(\vec{x}_{k,n}) \geq f(\vec{x}_{k+1,n+1}) \geq \vec{x}_{k,1}$ we get next simplex $S_{k+1}$ with new vertex $\vec{x}_{k+1,n+1}$ and n vertexes $\vec{x}_{k,i} \in S_k$ 

If $f(\vec{x}_{k+1,n+1}) > f(\vec{x}_{k,n}) \geq \vec{x}_{k,1}$ we compress the simplex.

If $\vec{x}_{k,1} > f(\vec{x}_{k+1,n+1})$ we stretch the simplex using the formula:
$$
f(\vec{x^*}_{k+1,n+1}) = \frac{1 - \beta}{n}\sum^n_{i=1}\vec{x}_{k,i} + \beta\vec{x}_{k+1,n+1}
$$

Where $\beta > 1$ - coefficient.

After the stretch if $f(\vec{x^*}_{k+1,n+1}) < \vec{x}_{k,1}$  

In [144]:
from operator import itemgetter

class Simplex:
    def __init__(self, vertexes, f):
        self.f = f
        self.V = sorted(
            [(v, f(v)) for v in vertexes], 
            key=itemgetter(1)
        )
        self.n = len(vertexes) - 1
    
    def insert(self, xnew, f_new, i):
        self.V = [self.V[k] if k != i else (xnew, f_new) for k in range(len(self.V))]
    
    def min(self):
        return self.V[0]
    
    def max(self):
        return self.V[self.n]
    
    def max2(self):
        return self.V[self.n-1]
    
    def sort(self):
        self.V = sorted(self.V, key=itemgetter(1))
    
    def reflect(self, x, alpha):
        vs =  [v for v, f_v in self.V if (v != x).any()]
        new_x = np.divide(1 + alpha, self.n) * sum(vs) - alpha*x
        return new_x, self.f(new_x)
    
    def scale(self, x, beta):
        vs = [v for v, f_v in self.V if (v != x).any()]
        new_x = np.divide(1 - beta, self.n) * np.sum(vs) + beta*x
        return new_x, self.f(new_x)
    
    def reduction(self, delta):
        vertexes = [v for v, f_v in self.V]
        vertexes = [vertexes[0] + delta*(vertexes[i] - vertexes[0]) for i in range (1, self.n+1)]
        self.V = [self.V[0]] + [(v, self.f(v)) for v in vertexes]
    
    def stop_check(self, epsilon):
        for i in range(self.n+1):
            a, b = 0, 0
            if i == self.n:
                a, b = self.V[i][0], self.V[0][0]
            else:
                a, b = self.V[i][0], self.V[i+1][0]
            if norm(a - b) > epsilon:
                return False
        return True
    
    def print(self):
        print("Points:", *[v for v, f_v in self.V], "Func vals:", *[f_v for v, f_v in self.V])

def first_vs(base, l):
    n = len(base)
    return [base] + [base + l*np.eye(n)[i] for i in range(0, n)]
    
def nelder_mead(f, x0, l, epsilon, alpha=1, beta=2, gamma=0.5, delta=0.5):
    smplx = Simplex(first_vs(x0, l), f)
    smplx.print()
    n = len(x0)
    k = 1
    f_ev = n + 1
    while not smplx.stop_check(epsilon):
        xm, fm = smplx.min()
        xh, fh = smplx.max() 
        xg, fg = smplx.max2()
        new_x, new_f = smplx.reflect(xh, alpha)
        f_ev += 1
        if new_f < fm:
            str_x, str_f = smplx.scale(new_x, beta)
            f_ev += 1
            if str_f < new_f:
                smplx.insert(str_x, str_f, smplx.n)
            else:
                smplx.insert(new_x, new_f, smplx.n)
        elif new_f < fg:
            smplx.insert(new_x, new_f, smplx.n)
        elif new_f < fh:
            smplx.insert(new_x, new_f, smplx.n)
            com_x, com_f = smplx.scale(new_x, gamma)
            f_ev += 1
            if com_f < new_f:
                smplx.insert(com_x, com_f, smplx.n)
            else: 
                smplx.reduction(delta)
                f_ev += n
        else:
            com_x, com_f = smplx.scale(xh, gamma)
            if com_f < fh:
                smplx.insert(com_x, com_f, smplx.n)
            else: 
                smplx.reduction(delta)
                f_ev += n
            
        smplx.sort()
        smplx.print()
        k += 1
    return *smplx.min(), k, f_ev
        
    

In [143]:
xm, fm, k, f_ev = nelder_mead(bukin, np.array([-10.5, 1.5]), 1, 1e-4)
print(f"""
x minimum: {xm},
f minimum: {fm},
Number of iterations: {k},
Number of function evaluations: {f_ev}
""")

Points: [-10.5   1.5] [-9.5  1.5] [-10.5   2.5] Func vals: 63.05260106459245 77.30312416870153 118.22090417536889
Points: [-10.5   1.5] [-9.5  0.5] [-9.5  1.5] Func vals: 63.05260106459245 63.4478877022476 77.30312416870153
Points: [-10.   1.] [-10.5   1.5] [-10.    1.5] Func vals: 0.0 63.05260106459245 70.71067811865476
Points: [-10.   1.] [-10.5   1. ] [-10.5   1.5] Func vals: 0.0 32.02062118716425 63.05260106459245
Points: [-10.   1.] [-10.25   1.  ] [-10.25   1.25] Func vals: 0.0 22.502499999999984 44.65392774872939
Points: [-10.   1.] [-10.125   1.   ] [-10.125   1.125] Func vals: 0.0 15.861971925561898 31.599311649411344
Points: [-10.   1.] [-10.0625   1.    ] [-10.0625   1.0625] Func vals: 0.0 11.198420541980537 22.3525684278096
Points: [-10.   1.] [-10.03125   1.     ] [-10.03125   1.03125] Func vals: 0.0 7.912180563232652 15.808612337427148
Points: [-10.   1.] [-10.015625   1.      ] [-10.015625   1.015625] Func vals: 0.0 5.592509427554237 11.179404254114544
Points: [-10.   1.